In [1]:
import pandas as pd
from dont_patronize_me import DontPatronizeMe
from ast import literal_eval
from torch.utils.data import DataLoader, Dataset
import os.path as osp
from typing import Optional
from transformer import RoBERTa

import pandas as pd
import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader, Dataset
from transformers.models.auto.tokenization_auto import AutoTokenizer
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = 'cuda'

In [2]:
class DPMDataset_submission(Dataset):
    def __init__(self, model: str, data) -> None:
        super().__init__()

        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.max_len = 512
        self.data = data

    def __getitem__(self, index):
        text = self.data[index]
        text = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True,
        )
        return {
            "ids": torch.tensor(text["input_ids"], dtype=torch.long),
            "mask": torch.tensor(text["attention_mask"], dtype=torch.long),
        }

    def __len__(self):
        return len(self.data)

In [3]:
dpm = DontPatronizeMe('../dataset', test_path="../dataset/task4_test.tsv")
dpm.load_test()

In [4]:
dataset_ = DPMDataset_submission(model = "mrm8488/distilroberta-finetuned-tweets-hate-speech",data=dpm.test_set)

In [5]:
data_loader = DataLoader(dataset_, batch_size=8)

In [13]:
best_model_path = "models/distillroberta_7/bert-val_loss0.47.ckpt"
hf_name = "mrm8488/distilroberta-finetuned-tweets-hate-speech"
module = RoBERTa(model=hf_name).load_from_checkpoint(best_model_path, model=hf_name)


mrm8488/distilroberta-finetuned-tweets-hate-speech
2048
mrm8488/distilroberta-finetuned-tweets-hate-speech
2048


TypeError: forward() missing 1 required positional argument: 'mask'

In [14]:
module.eval()
module.freeze()
preds = []
file_path = "../submissions/task2.txt"
for batch in data_loader:
    ids,mask = batch["ids"], batch["mask"]
    out = module(ids,mask)
    predictions = torch.argmax(out, 1).numpy()
    for i, pred in enumerate(predictions):
        preds.append(pred)
    
    break
result_df = pd.DataFrame(preds, columns=None)
result_df.to_csv(file_path, index=False)
print("submission created")

submission created


In [18]:
module.eval()
module.freeze()
preds = []
sigmoid = torch.nn.Sigmoid()
file_path = "../submissions/task2.txt"
for batch in data_loader:
    ids,mask = batch["ids"], batch["mask"]
    out = module(ids,mask)
    print(out)
    # predictions = torch.argmax(out, 1).numpy()
    predictions = sigmoid(out)
    print(predictions)
    for i, pred in enumerate(predictions):
        preds.append(pred)
    
    break

tensor([[  2.8737,   0.7811,  -0.6697,  -3.2605,  -3.0758,  -1.5863,  -6.0160],
        [  4.8789,   0.4824,  -1.8092,  -2.2792,  -1.2954,  -2.4269,  -4.2241],
        [ -1.7420,  -3.1297,  -1.0128,  -3.8017,  -0.6052,   4.0099,  -6.2521],
        [  1.6155,  -0.8541,  -4.0080,  -2.8658,  -1.7835,  -2.0147,  -8.0312],
        [  0.6495,  -4.7476,  -0.6911,  -1.3976,  -3.1453,  -0.5175,  -6.0527],
        [ -2.4466,  -1.9455,   4.2553,   0.5320,  -4.6862,  -4.6971, -11.6382],
        [  1.4569,  -1.7137,  -2.1345,  -4.1526,  -3.9218,  -0.3879,  -6.3081],
        [  2.4493,  -2.0093,  -0.7099,   0.6080,  -2.2352,   0.5258,  -4.6004]])
tensor([[9.4653e-01, 6.8591e-01, 3.3857e-01, 3.6950e-02, 4.4115e-02, 1.6990e-01,
         2.4334e-03],
        [9.9245e-01, 6.1831e-01, 1.4073e-01, 9.2863e-02, 2.1494e-01, 8.1145e-02,
         1.4427e-02],
        [1.4906e-01, 4.1900e-02, 2.6644e-01, 2.1846e-02, 3.5315e-01, 9.8219e-01,
         1.9226e-03],
        [8.3417e-01, 2.9858e-01, 1.7845e-02, 5.387